### Import các thư viện cần thiết

In [ ]:
import os
import math
import torch
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from torch import nn
from PIL import Image
from tqdm import tqdm
from diffusers import AutoencoderKL
from torch.nn import functional as F
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from transformers import CLIPTokenizer, CLIPTextModel
import torch.optim.lr_scheduler as lr_scheduler
from torch.amp import GradScaler, autocast

### Cố định tham số ngẫu nhiên

In [ ]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ["PYTHONHASHSEED"] = str(seed)
    print(f"Seed set to {seed}")

set_seed()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Tải bộ dữ liệu

In [ ]:
# https://drive.google.com/file/d/15Z_F4Dwgb3NLqEGnVMUEJqyxXgW7Gx-h/view?usp=sharing
!gdown 15Z_F4Dwgb3NLqEGnVMUEJqyxXgW7Gx-h
!unzip blobs_crawled_data.zip

### Xây dựng các class Attention

In [ ]:
class SelfAttention(nn.Module):
    def __init__(self, num_attn_heads, hidden_dim, in_proj_bias=True, out_proj_bias=True):
        super().__init__()
        self.num_heads = num_attn_heads
        self.head_size = hidden_dim // num_attn_heads
        self.qkv_proj = nn.Linear(hidden_dim, 3 * hidden_dim, bias=in_proj_bias)
        self.output_proj = nn.Linear(hidden_dim, hidden_dim, bias=out_proj_bias)

    def forward(self, features, use_causal_mask=False):
        b, s, d = features.shape
        qkv_combined = self.qkv_proj(features)
        q_mat, k_mat, v_mat = torch.chunk(qkv_combined, 3, dim=-1)
        q_mat = q_mat.view(b, s, self.num_heads, self.head_size).permute(0, 2, 1, 3)
        k_mat = k_mat.view(b, s, self.num_heads, self.head_size).permute(0, 2, 1, 3)
        v_mat = v_mat.view(b, s, self.num_heads, self.head_size).permute(0, 2, 1, 3)

        qk = torch.matmul(q_mat, k_mat.transpose(-2,-1))
        sqrt_qk = qk / math.sqrt(self.head_size)

        if use_causal_mask:
            causal_mask = torch.triu(torch.ones_like(sqrt_qk, dtype=torch.bool), diagonal=1)
            sqrt_qk = sqrt_qk.masked_fill(causal_mask,-torch.inf)

        attn_weights = torch.softmax(sqrt_qk, dim=-1)
        attn_values = torch.matmul(attn_weights, v_mat)

        attn_values = attn_values.permute(0, 2, 1, 3).contiguous()
        attn_values = attn_values.view(b, s, d)

        final_output = self.output_proj(attn_values)
        return final_output

In [ ]:
class CrossAttention(nn.Module):
    def __init__(self, num_attn_heads, query_dim, context_dim, in_proj_bias=True, out_proj_bias=True):
        super().__init__()
        self.num_heads = num_attn_heads
        self.head_size = query_dim // num_attn_heads

        self.query_map = nn.Linear(query_dim, query_dim, bias=in_proj_bias)
        self.key_map = nn.Linear(context_dim, query_dim, bias=in_proj_bias)
        self.value_map = nn.Linear(context_dim, query_dim, bias=in_proj_bias)

        self.output_map = nn.Linear(query_dim, query_dim, bias=out_proj_bias)

    def forward(self, query_input, context_input):
        b_q, s_q, d_q = query_input.shape
        _, s_kv, _ = context_input.shape

        q_mat = self.query_map(query_input)
        k_mat = self.key_map(context_input)
        v_mat = self.value_map(context_input)

        q_mat = q_mat.view(b_q, s_q, self.num_heads, self.head_size).permute(0, 2, 1, 3)
        k_mat = k_mat.view(b_q, s_kv, self.num_heads, self.head_size).permute(0, 2, 1, 3)
        v_mat = v_mat.view(b_q, s_kv, self.num_heads, self.head_size).permute(0, 2, 1, 3)

        qk = torch.matmul(q_mat, k_mat.transpose(-2,-1))
        sqrt_qk = qk / math.sqrt(self.head_size)
        attn_weights = torch.softmax(sqrt_qk, dim=-1)

        attn_values = torch.matmul(attn_weights, v_mat)
        attn_values = attn_values.permute(0, 2, 1, 3).contiguous()
        attn_values = attn_values.view(b_q, s_q, d_q)

        final_output = self.output_map(attn_values)
        return final_output

### Khai báo class DDPM

In [ ]:
class DDPMScheduler:
    def __init__(
        self,
        random_generator,
        train_timesteps=1000,
        diffusion_beta_start=0.00085,
        diffusion_beta_end=0.012
    ):
    self.betas = torch.linspace(
        diffusion_beta_start ** 0.5, diffusion_beta_end ** 0.5, train_timesteps,
        dtype=torch.float32) ** 2
    self.alphas = 1.0-self.betas
    self.alphas_cumulative_product = torch.cumprod(self.alphas, dim=0)
    self.one_val = torch.tensor(1.0)
    self.prng_generator = random_generator
    self.total_train_timesteps = train_timesteps
    self.schedule_timesteps = torch.from_numpy(np.arange(0, train_timesteps)[::-1].copy())

    def set_steps(self, num_sampling_steps=50):
        self.num_sampling_steps = num_sampling_steps
        step_scaling_factor = self.total_train_timesteps // self.num_sampling_steps
        timesteps_for_sampling = (
            np.arange(0, num_sampling_steps) * step_scaling_factor
        ).round()[::-1].copy().astype(np.int64)
        self.schedule_timesteps = torch.from_numpy(timesteps_for_sampling)

    def _get_prior_timestep(self, current_timestep):
        previous_t = current_timestep - self.total_train_timesteps//self.num_sampling_steps
        return previous_t

    def _calculate_variance(self, timestep):
        prev_t = self._get_prior_timestep(timestep)
        alpha_cumprod_t = self.alphas_cumulative_product[timestep]
        alpha_cumprod_t_prev = self.alphas_cumulative_product[prev_t] if prev_t >= 0 else self.one_val
        beta_t_current = 1-alpha_cumprod_t / alpha_cumprod_t_prev
        variance_value = (1-alpha_cumprod_t_prev) / (1-alpha_cumprod_t) * beta_t_current
        variance_value = torch.clamp(variance_value, min=1e-20)
        return variance_value

    def adjust_strength(self, strength_level=1):
        initial_step_index = self.num_sampling_steps-int(self.num_sampling_steps * strength_level)
        self.schedule_timesteps = self.schedule_timesteps[initial_step_index:]
        self.start_sampling_step = initial_step_index

    def step(self, current_t, current_latents, model_prediction):
        t = current_t
        prev_t = self._get_prior_timestep(t)

        alpha_cumprod_t = self.alphas_cumulative_product[t]
        alpha_cumprod_t_prev = self.alphas_cumulative_product[prev_t] if prev_t >= 0 else self.one_val
        beta_cumprod_t = 1-alpha_cumprod_t
        beta_cumprod_t_prev = 1-alpha_cumprod_t_prev
        alpha_t_current = alpha_cumprod_t / alpha_cumprod_t_prev
        beta_t_current = 1-alpha_t_current

        predicted_original = (current_latents-beta_cumprod_t**0.5 * model_prediction) / alpha_cumprod_t**0.5

        original_coeff = (alpha_cumprod_t_prev ** 0.5 * beta_t_current) / beta_cumprod_t
        current_coeff = alpha_t_current**0.5 * beta_cumprod_t_prev / beta_cumprod_t

        predicted_prior_mean = original_coeff*predicted_original + current_coeff*current_latents

        variance_term = 0
        if t > 0:
            target_device = model_prediction.device
            noise_component = torch.randn(
                model_prediction.shape,
                generator=self.prng_generator,
                device=target_device,
                dtype=model_prediction.dtype
            )
            variance_term = (self._calculate_variance(t) ** 0.5) * noise_component

        predicted_prior_sample = predicted_prior_mean + variance_term
        return predicted_prior_sample

    def add_noise(self, initial_samples, noise_timesteps):
        alphas_cumprod = self.alphas_cumulative_product.to(
            device=initial_samples.device,
            dtype=initial_samples.dtype
        )
        noise_timesteps = noise_timesteps.to(initial_samples.device)
        sqrt_alpha_cumprod = alphas_cumprod[noise_timesteps] ** 0.5
        sqrt_alpha_cumprod = sqrt_alpha_cumprod.view(
            sqrt_alpha_cumprod.shape[0], *([1] * (initial_samples.ndim-1))
        )
        sqrt_one_minus_alpha_cumprod = (1-alphas_cumprod[noise_timesteps]) ** 0.5
        sqrt_one_minus_alpha_cumprod = sqrt_one_minus_alpha_cumprod.view(
        sqrt_one_minus_alpha_cumprod.shape[0], *([1] * (initial_samples.ndim- 1)))
        random_noise = torch.randn(
            initial_samples.shape, generator=self.prng_generator,
            device=initial_samples.device, dtype=initial_samples.dtype
        )
        noisy_result = sqrt_alpha_cumprod*initial_samples + sqrt_one_minus_alpha_cumprod*random_noise
        return noisy_result, random_noise

In [ ]:
class UNET_ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, time_dim=1280):
        super().__init__()
        self.gn_feature = nn.GroupNorm(32, in_channels)
        self.conv_feature = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.time_embedding_proj = nn.Linear(time_dim, out_channels)

        self.gn_merged = nn.GroupNorm(32, out_channels)
        self.conv_merged = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)

        if in_channels == out_channels:
            self.residual_connection = nn.Identity()
        else:
            self.residual_connection = nn.Conv2d(in_channels, out_channels, kernel_size=1, padding=0)

    def forward(self, input_feature, time_emb):
        residual = input_feature
        h = self.gn_feature(input_feature)
        h = F.silu(h)
        h = self.conv_feature(h)

        time_emb_processed = F.silu(time_emb)
        time_emb_projected = self.time_embedding_proj(time_emb_processed)
        time_emb_projected = time_emb_projected.unsqueeze(-1).unsqueeze(-1)

        merged_feature = h + time_emb_projected
        merged_feature = self.gn_merged(merged_feature)
        merged_feature = F.silu(merged_feature)
        merged_feature = self.conv_merged(merged_feature)

        output = merged_feature + self.residual_connection(residual)
        return output

In [ ]:
class UNET_AttentionBlock(nn.Module):
    def __init__(self, num_heads, head_dim, context_dim=512):
        super().__init__()
        embed_dim = num_heads * head_dim
        self.gn_in = nn.GroupNorm(32, embed_dim, eps=1e-6)
        self.proj_in = nn.Conv2d(embed_dim, embed_dim, kernel_size=1, padding=0)
        self.ln_1 = nn.LayerNorm(embed_dim)
        self.attn_1 = SelfAttention(num_heads, embed_dim, in_proj_bias=False)
        self.ln_2 = nn.LayerNorm(embed_dim)
        self.attn_2 = CrossAttention(num_heads, embed_dim, context_dim, in_proj_bias=False)
        self.ln_3 = nn.LayerNorm(embed_dim)
        self.ffn_geglu = nn.Linear(embed_dim, 4 * embed_dim * 2)
        self.ffn_out = nn.Linear(4 * embed_dim, embed_dim)
        self.proj_out = nn.Conv2d(embed_dim, embed_dim, kernel_size=1, padding=0)

    def forward(self, input_tensor, context_tensor):
        skip_connection = input_tensor

        B, C, H, W = input_tensor.shape
        HW = H * W

        h = self.gn_in(input_tensor)
        h = self.proj_in(h)
        h = h.view(B, C, HW).transpose(-1,-2)

        attn1_skip = h
        h = self.ln_1(h)
        h = self.attn_1(h)
        h = h + attn1_skip

        attn2_skip = h
        h = self.ln_2(h)
        h = self.attn_2(h, context_tensor)
        h = h + attn2_skip

        ffn_skip = h
        h = self.ln_3(h)
        intermediate, gate = self.ffn_geglu(h).chunk(2, dim=-1)
        h = intermediate * F.gelu(gate)
        h = self.ffn_out(h)
        h = h + ffn_skip

        h = h.transpose(-1,-2).view(B, C, H, W)
        output = self.proj_out(h) + skip_connection
        return output

In [ ]:
class Upsample(nn.Module):
    def __init__(self, num_channels):
        super().__init__()
        self.conv = nn.Conv2d(num_channels, num_channels, kernel_size=3, padding=1)

    def forward(self, feature_map):
        x = F.interpolate(feature_map, scale_factor=2, mode=’nearest’)
        x = self.conv(x)
        return x

In [ ]:
class SwitchSequential(nn.Sequential):
    def forward(self, x, guidance_context, time_embedding):
        for module_instance in self:
            if isinstance(module_instance, UNET_AttentionBlock):
                x = module_instance(x, guidance_context)
            elif isinstance(module_instance, UNET_ResidualBlock):
                x = module_instance(x, time_embedding)
            else:
                x = module_instance(x)
        return x

class TimeEmbedding(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.proj1 = nn.Linear(n_embd, 4 * n_embd)
        self.proj2 = nn.Linear(4 * n_embd, 4 * n_embd)

    def forward(self, x):
        x = self.proj1(x)
        x = F.silu(x)
        x = self.proj2(x)
        return x

### Xây dựng hàm mã hóa thông tin thời gian

In [ ]:
def embed_a_timestep(timestep, embedding_dim=320):
    half_dim = embedding_dim // 2
    freqs = torch.exp(-math.log(10000) *
                      torch.arange(start=0, end=half_dim, dtype=torch.float32)
                      / half_dim)
    x = torch.tensor([timestep], dtype=torch.float32)[:, None] * freqs[None]
    return torch.cat([torch.cos(x), torch.sin(x)], dim=-1)

def embed_timesteps(timesteps, embedding_dim=320):
    half_dim = embedding_dim // 2
    freqs = torch.exp(-math.log(10000) *
                      torch.arange(half_dim, dtype=torch.float32) /
                      half_dim).to(device=timesteps.device)
    args = timesteps[:, None].float() * freqs[None, :]
    return torch.cat([torch.cos(args), torch.sin(args)], dim=-1)

### Khai báo Diffusion model

In [ ]:
class Diffusion(nn.Module):
    def __init__(self, h_dim=128, n_head=4):
        super().__init__()
        self.time_embedding = TimeEmbedding(320)
        self.unet = UNET(h_dim, n_head)
        self.unet_output = UNETOutputLayer(h_dim, 4)

    @torch.autocast(
        device_type='cuda', dtype=torch.float16,
        enabled=True, cache_enabled=True
    )
    def forward(self, latent, context, time):
        time = self.time_embedding(time)
        output = self.unet(latent, context, time)
        output = self.unet_output(output)
        return output

### Khai báo mô hình CLIP

In [ ]:
class CLIPTextEncoder(nn.Module):
    def __init__(self):
        super().__init__()
        CLIP_id = "openai/clip-vit-base-patch32"
        self.tokenizer = CLIPTokenizer.from_pretrained(CLIP_id)
        self.text_encoder = CLIPTextModel.from_pretrained(CLIP_id)
        self.device = "cuda" if torch.cuda.is_available() else "cpu"

        for param in self.text_encoder.parameters():
            param.requires_grad = False

        self.text_encoder.eval()
        self.text_encoder.to(self.device)

    def forward(self, prompts):
        inputs = self.tokenizer(
            prompts,
            padding="max_length",
            truncation=True,
            max_length=self.text_encoder.config.max_position_embeddings,
            return_tensors="pt"
        )
        input_ids = inputs.input_ids.to(self.device)
        attention_mask = inputs.attention_mask.to(self.device)

        with torch.no_grad():
            text_encoder_output = self.text_encoder(
                input_ids=input_ids,
                attention_mask=attention_mask
            )
        last_hidden_states = text_encoder_output.last_hidden_state
        return last_hidden_states

### Khai báo mô hình pre-trained VAE

In [ ]:
VAE_id = "stabilityai/sd-vae-ft-mse"
vae = AutoencoderKL.from_pretrained(VAE_id)
vae.requires_grad_(False)
vae.eval()

### Khai báo hàm trực quan hóa ảnh và scale giá trị ảnh

In [ ]:
def show_images(images, title="", titles=[]):
    plt.figure(figsize=(8, 8))
    for i in range(min(25, len(images))):
        plt.subplot(5, 5, i+1)
        img = images[i].permute(1, 2, 0).cpu().numpy()
        plt.imshow(img)
        if titles:
            plt.title(titles[i])
        plt.axis("off")
    plt.suptitle(title)
    plt.tight_layout()
    plt.show()

def rescale(value, in_range, out_range, clamp=False):
    in_min, in_max = in_range
    out_min, out_max = out_range

    in_span = in_max-in_min
    out_span = out_max-out_min

    scaled_value = (value-in_min) / (in_span + 1e-8)
    rescaled_value = out_min + (scaled_value * out_span)

    if clamp:
        rescaled_value = torch.clamp(
            rescaled_value,
            out_min, out_max
        )

    return rescaled_value

###  Khai báo class PyTorch dataset

In [ ]:
WIDTH, HEIGHT = 32, 32
batch_size = 32

class EmojiDataset(Dataset):
    def __init__(self, csv_files, image_folder, transform=None):
        self.dataframe = pd.concat([pd.read_csv(csv_file) for csv_file in csv_files])
        self.images_folder = image_folder
        self.dataframe["image_path"] = self.dataframe["file_name"].str.replace("\\", "/")
        self.image_paths = self.dataframe["image_path"].tolist()
        self.titles = self.dataframe["prompt"].tolist()
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        image_path = self.images_folder + "/" + self.image_paths[idx]
        title = self.titles[idx]
        title = title.replace('"', "").replace("'", "")
        image = Image.open(image_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, title

In [ ]:
transform = transforms.Compose([
    transforms.Resize(
        (WIDTH, HEIGHT),
        interpolation=transforms.InterpolationMode.BICUBIC
    ),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

csv_files = ['/content/blobs_crawled_data/metadata.csv']
image_folder = '/content/blobs_crawled_data/images'

train_dataset = EmojiDataset(
    csv_files=csv_files,
    image_folder=image_folder,
    transform=transform
)
train_dataloader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=2,
    pin_memory=True,
    persistent_workers=True
)

### Thực hiện huấn luyện

In [ ]:
def train(diffusion, vae, text_encoder, scheduler,
          optimizer, lr_scheduler, scaler,
          criterion, dataloader, num_epochs, device="cuda"):
    losses = []
    for epoch in range(num_epochs):
        diffusion.train()

        epoch_loss = 0.0
        progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}",
                            leave=False)

        for batch_idx, (images, titles) in enumerate(progress_bar):
            images = images.to(device)
            image_titles = [f"A photo of {title}" for title in titles]
            image_titles = [title if random.random() < 0.5 else "" for title in image_titles]
            with torch.no_grad():
                latents = vae.encode(images).latent_dist.sample() * 0.18215

            timesteps = torch.randint(
                0, scheduler.total_train_timesteps,
                 (latents.shape[0],), device=device
                )

            noisy_latents, noise = scheduler.add_noise(latents, timesteps)
            time_embeddings = embed_timesteps(timesteps).to(device)
            text_embeddings = text_encoder(image_titles)

            noise_pred = diffusion(noisy_latents, text_embeddings, time_embeddings)

            with autocast(device_type="cuda", dtype=torch.float16,
                          enabled=True, cache_enabled=True):
                loss = criterion(noise_pred, noise)

            optimizer.zero_grad()
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            batch_loss = loss.item()
            epoch_loss += batch_loss

            progress_bar.set_postfix(loss=f"{batch_loss:.5f}",
                                     lr=f"{optimizer.param_groups[0]['lr']:.6f}")

        lr_scheduler.step()
        avg_epoch_loss = epoch_loss / len(dataloader)
        if (epoch + 1) % 10 == 0 or epoch == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}]-Avg Loss: {avg_epoch_loss:.5f}")

        losses.append(avg_epoch_loss)

    print("Training finished!")
    return losses

In [ ]:
EPOCHS = 300

h_dim = 384
n_head = 8

vae = vae.to(device)
diffusion = Diffusion(h_dim, n_head).to(device)
clip = CLIPTextEncoder().to(device)

random_generator = torch.Generator(device="cuda")
noise_scheduler = DDPMScheduler(random_generator)

optimizer = torch.optim.AdamW(diffusion.parameters(), lr=1e-4)
criterion = torch.nn.MSELoss()

lrate_scheduler = lr_scheduler.CosineAnnealingLR(
    optimizer, T_max=EPOCHS, eta_min=1e-5
)
scaler = GradScaler()

def count_parameters(model):
    return sum(p.numel() for p in model.parameters())

vae_params = count_parameters(vae)
diffusion_params = count_parameters(diffusion)
clip_params = count_parameters(clip)

print(f"VAE parameters: {vae_params}")
print(f"Diffusion parameters: {diffusion_params}")
print(f"CLIP parameters: {clip_params}")
print(f"Total parameters: {vae_params + diffusion_params + clip_params}")

losses = train(diffusion, vae, clip, noise_scheduler,
               optimizer, lrate_scheduler, scaler,
               criterion, train_dataloader, EPOCHS, device=device)

In [ ]:
torch.save(diffusion.state_dict(), "Emoji_SD.pth")
!zip-r Emoji_SD.zip Emoji_SD.pth

## Triển khai mô hình

### Xây dựng hàm sinh ảnh

In [ ]:
LATENTS_WIDTH = WIDTH // 8
LATENTS_HEIGHT = HEIGHT // 8

def generate_image(
    prompt,
    diffusion,
    vae,
    text_encoder,
    scheduler,
    num_inference_steps=100,
    seed=None,
    device="cuda" if torch.cuda.is_available() else "cpu",
):
    rng_generator = torch.Generator(device=device)
    if seed is None:
        rng_generator.seed()
    else:
        rng_generator.manual_seed(seed)

    prompts = [prompt]
    text_embeddings = text_encoder(prompts).to(device)

    scheduler.set_steps(num_inference_steps)

    latent_shape = (1, 4, LATENTS_HEIGHT, LATENTS_WIDTH)

    noisy_latents = torch.randn(
        latent_shape, generator=rng_generator, device=device)
    timesteps = scheduler.schedule_timesteps

    for t in tqdm(timesteps):
        latent_model_input = noisy_latents
        time_embedding = embed_a_timestep(t).to(device)
        with torch.no_grad():
            noise_pred = diffusion(
                latent_model_input,
                text_embeddings,
                time_embedding
            )
        noisy_latents = scheduler.step(
            t,
            noisy_latents,
            noise_pred
        )

    final_latents = noisy_latents / 0.18215
    with torch.no_grad():
        decoded_image_tensor = vae.decode(final_latents).sample
    image_output = rescale(decoded_image_tensor, (-1, 1), (0, 255), clamp=True)
    image_output = image_output.permute(0, 2, 3, 1).to("cpu", torch.uint8).numpy()
 return image_output[0]